In [1]:
import os

In [2]:
%pwd

'c:\\Users\\rahul\\Desktop\\Project\\StockForcastingAI\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\rahul\\Desktop\\Project\\StockForcastingAI'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_folder: Path
    unzip_dir: Path

In [6]:
from src.financeProject.constants import *
from src.financeProject.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data(self):
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_url = config.source_url,
            local_data_folder = config.local_data_folder,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config

In [8]:
import os
import zipfile
import gdown
from src.financeProject import logger
from src.financeProject.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        try:
            dataset_url = self.config.source_url
            zip_download_dir = self.config.local_data_folder
            os.makedirs("artifacts/data_ingestion", exist_ok = True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = "https://drive.google.com/uc?/export=download&id="
            gdown.download(prefix + file_id, zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir

        os.makedirs(unzip_path, exist_ok = True)
        with zipfile.ZipFile(self.config.local_data_folder, "r") as zip_ref:
            zip_ref.extractall(unzip_path)

In [10]:
try:
    config = ConfigurationManager()
    data_ingestioin_config = config.get_data()
    data_ingestion = DataIngestion(config = data_ingestioin_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-06-25 18:33:15,148: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-06-25 18:33:15,149: INFO: common: yaml file: params.yaml loaded successfully]
[2024-06-25 18:33:15,149: INFO: common: created directory at: artifacts]
[2024-06-25 18:33:15,150: INFO: common: created directory at: artifacts/data_ingestion]
[2024-06-25 18:33:15,151: INFO: 3826238980: Downloading data from https://drive.google.com/file/d/1Ji1R6ryzVQn45-5cOA9Gak1Y7VFCCOXj/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From: https://drive.google.com/uc?/export=download&id=1Ji1R6ryzVQn45-5cOA9Gak1Y7VFCCOXj
To: c:\Users\rahul\Desktop\Project\StockForcastingAI\artifacts\data_ingestion\data.zip
100%|██████████| 415k/415k [00:00<00:00, 2.92MB/s]

[2024-06-25 18:33:16,959: INFO: 3826238980: Downloaded data from https://drive.google.com/file/d/1Ji1R6ryzVQn45-5cOA9Gak1Y7VFCCOXj/view?usp=sharing into file artifacts/data_ingestion/data.zip]
